In [1]:
import pandas as pd
import numpy as np
import theano
from sklearn import datasets
from sklearn.cross_validation import train_test_split

data_root = '/Users/rbekbolatov/repos/gh/bekbolatov/kaggle/events/liberty/cxxnet_proj'

iris = datasets.load_iris()
dat_x = pd.DataFrame(iris.data)
dat_y = iris.target

Using gpu device 0: GeForce GT 750M


Split dataset into train/test

In [2]:
train_index, test_index = train_test_split(range(dat_x.shape[0]), test_size=0.3, random_state=104)
train_x = dat_x.loc[train_index, :]
train_y = dat_y[train_index]
cv_x = dat_x.loc[test_index, :]
cv_y = dat_y[test_index]

In [ ]:
train_x = np.asarray(train_x, dtype = theano.config.floatX)
train_y = np.asarray(train_y, dtype = 'int32')
cv_x = np.asarray(cv_x, dtype = theano.config.floatX)
cv_y = np.asarray(cv_y, dtype = 'int32')

In [ ]:
from lasagne.layers import DenseLayer
from lasagne.layers import InputLayer
from lasagne.layers import DropoutLayer
from lasagne.updates import nesterov_momentum, sgd
from lasagne.objectives import squared_error, categorical_crossentropy, binary_crossentropy
from lasagne.nonlinearities import sigmoid, rectify, softmax, linear, tanh
from nolearn.lasagne import NeuralNet, TrainSplit

def on_epoch_finished(obj, train_history):
    if (len(train_history) > 20):
        inLastEight = any([h['valid_loss_best'] for h in  train_history[-8:-1]] +
                          [train_history[-1]['valid_loss'] < train_history[-2]['valid_loss'],
                           train_history[-1]['valid_loss'] < train_history[-3]['valid_loss'],
                           train_history[-2]['valid_loss'] < train_history[-3]['valid_loss'],
                           ])
        if not inLastEight:
            print("Stopping early")
            raise StopIteration

def NeuralNetConstructor(num_features, max_epochs=5, update_learning_rate=0.2, verbose=1):
    layers0 = [
        ('input', InputLayer),
#         ('dropout0', DropoutLayer),
        ('hidden1', DenseLayer),
#         ('hidden2', DenseLayer),
        ('output', DenseLayer)]

    net0 = NeuralNet(
        layers=layers0,

        input_shape=(None, num_features),
        input_nonlinearity=sigmoid,

#         dropout0_p=0.0,

        hidden1_num_units=7, #75, #20,
        hidden1_nonlinearity=tanh,

#         hidden2_num_units=3, #70, #24,
#         hidden2_nonlinearity=tanh,

        output_num_units=3,
        output_nonlinearity=softmax,

        on_epoch_finished=[on_epoch_finished],

        objective_loss_function=categorical_crossentropy, #binary_crossentropy, #squared_error, categorical_crossentropy
        update=nesterov_momentum,
        update_learning_rate=update_learning_rate, #0.02,
        update_momentum=0.9,
        train_split=TrainSplit(eval_size=0.2),
        verbose=verbose,
        regression=False,
        max_epochs=max_epochs)

    return net0

In [ ]:
network = NeuralNetConstructor(4, max_epochs=1000, update_learning_rate=0.01, verbose=0)

In [ ]:
network.fit(train_x, train_y)

In [ ]:
cv_preds = network.predict(cv_x)
pd.Series(cv_preds == cv_y).value_counts()